# K-Means Clustering Evaluation

This Python notebook is used for evaluation of a dictionary that is produced by:

- Find the cluster a word belongs to 
- Find the other words in the cluster for a specific word
- Compare between clusters

In [8]:
import pickle
import random

# Specify the files
FILE_DICT_500 = "C:/Users/MyPC/Desktop/Vegito/Word Dictionaries/dict_500C.pk"
FILE_CLUS_500 = "C:/Users/MyPC/Desktop/Vegito/K-Means Models/full_500C.pk"

# Load using pickle
array_dict_cluster_500 = pickle.load(open(FILE_DICT_500, "rb"))
word_centroid_map_500 =  pickle.load(open(FILE_CLUS_500,"rb"))

total_clusters_500 = max(word_centroid_map_500.values()) + 1

average_word_500 = round(len(word_centroid_map_500)/total_clusters_500)

In [35]:
# Find the cluster of words, based on a given word
search = input("ENTER WORD TO SEARCH: ")

# Get the key, or cluster number
# NOTE: Different clusters can have same results
cluster_num_500 = word_centroid_map_500[search]

# Return the array based on the cluster number
words_500 = array_dict_cluster_500[cluster_num_500]['word_list']

# Randomly subsample 200 words
words = random.sample(words_500, 200)

# Display results
print("SEARCHED WORD: %s \n" % (search))

print("TOTAL WORDS: %i \n" % (len(words_500)))

# Pretty Printing
for i, word in enumerate(words):
    
    if i % 5 != 0:
        print(word, " ", end="")
    else:
        print("\n")

ENTER WORD TO SEARCH: doflamingo
SEARCHED WORD: doflamingo 

TOTAL WORDS: 2020 



laffitte  juzuo  kataras  kenpachi  

isshins  hecuo  vali  rokushiki  

majuub  recoome  byakugan  goshinki  

unohanas  radditz  torquasm  kido  

yammamoto  tessai  iihiko  tagashi  

kanassans  saiyans  sandbend  reaitsu  

haoshoku  bardock  hollowification  kishin  

shabody  kyubii  dressrosa  uroge  

tamakoma  sanin  bellemere  hirudegarn  

chlammy  nougami  tenryuubito  deidera  

geppou  hyousube  kacchan  hokage  

shabondy  gaibon  prepgod  koromon  

tullece  sasakibe  justsu  mutaito  

teridax  kyros  tohja  shokan  

genos  ukitakes  zancrow  dedoria  

fraccion  plotkai  kuivira  jinbe  

tosen  tenrou  rukias  genryuusai  

budou  luffi  zommari  piccolo  

ussopp  fough  wanze  destructo  

polybotes  majjin  ywachs  waterbend  

zorc  bikaku  krilin  grandline  

lucci  butai  septar  sandai  

nrvnqsr  daizenshuu  kaioshins  nen  

dorry  rubberyness  yamacha  gaang  

nozarashi  s